--------------------------------------------SCRIPT OVERVIEW---------------------------


This script connects to the Community Google Sheet using the gspread API, authenticates through locally configured credentials, extracts the headers, and creates a Pandas DataFrame.
It then loads this DataFrame into the Bronze layer of the SQL Server as a JSON payload.

Additionally, the script generates essential metadata used for tracking and monitoring incremental loads.

NOTE:
--Credentials are securely stored as local environment variables,they are not shared in the repository.
--The API fetches only the latest updates (incremental loads), ensuring minimal duplication.
--Make sure your environment variables are properly configured before running this script, otherwise authentication will fail.
--This script only handles the Bronze layer ingestion that means cleaning, transformations, and schema modeling occur in later stages (Silver/Gold).

In [1]:
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [2]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [3]:

#googlesheet credentials from environment variables

UGASHEET_CREDENTIALS = os.getenv("UGASHEET_CRED_PATH")
SHEET_KEY = os.getenv("UGASHEET_KEY")
WORKSHEET_NAME = os.getenv("UGASHEET_NAME", "atendees")
DATABASE_URL = os.getenv("DB_URL")



In [4]:

# connecting to the google sheet using the credentials
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(UGASHEET_CREDENTIALS, scope)
gc = gspread.authorize(creds) #authorize the gspread API client to read the data
sh = gc.open_by_key(SHEET_KEY) #open the google sheet by its key
ws = sh.worksheet(WORKSHEET_NAME) #selecting the worksheet



In [5]:
import json, hashlib, pandas as pd, uuid
from datetime import datetime



In [6]:
data = ws.get_all_values()#get all data from the worksheet
headers = data.pop(0)#extract the headers since the first row contains headers on the data table in the google sheet
df = pd.DataFrame(data, columns=headers) #create a pandas dataframe from the data and headers

In [ ]:
#function to create a unique hash for each row
def row_hash(r):
    rs = json.dumps(r, sort_keys=True,default=str)
    return hashlib.sha256(rs.encode('utf-8')).hexdigest()

#adding metadata columns to the dataframe
df['ingested_at'] = datetime.utcnow() #timestamp for when the data was ingested
df['source_sheet'] = WORKSHEET_NAME
df['source_row'] = df.index + 2  #row number in the google sheet (adding 2 to account for header row and 0-based index)
df['row_hash'] = df.apply(lambda r: row_hash(r.to_dict()), axis=1) #unique hash for each row
df['ingest_batch_id'] = str(uuid.uuid4()) #generate a unique batch id for each ingestion run
df['payload'] = df.apply(lambda r: json.dumps(r.drop(['row_hash','ingested_at','ingest_batch_id','source_sheet','source_row']).to_dict(), default=str), axis=1) #json payload of the entire row


In [8]:
engine = create_engine(DATABASE_URL)

In [13]:
with engine.begin() as conn: 
    for idx, row in df.iterrows():
        exists = conn.execute(
            text("SELECT 1 FROM bronze.ugacooq_raw WHERE row_hash = :row_hash"),
            {"row_hash": row['row_hash']}).fetchone()  
            
        if not exists:
            conn.execute(
                text("""   
                INSERT INTO bronze.ugacooq_raw (ingested_at, source_sheet, source_row, payload, row_hash, ingest_batch_id)
                VALUES (:ingested_at, :source_sheet, :source_row, :payload, :row_hash, :ingest_batch_id)
                """),
                {
                    "ingested_at": row['ingested_at'],
                    "source_sheet": row['source_sheet'],
                    "source_row": idx + 2,  # Google Sheets rows start at 2 (header = row 1)
                    "payload": row['payload'],
                    "row_hash": row['row_hash'],
                    "ingest_batch_id": row['ingest_batch_id']
                }
        )
